In [1]:
import warnings
warnings.filterwarnings("ignore")
import cv2
import numpy as np
import tensorflow as tf
import urllib.request
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import os
import pandas as pd
import joblib
from datetime import datetime
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# os.chdir(r'C:\Users\MBathija\BI_Engineering\Image_angle')
import importlib
import image_angle_func_updated
importlib.reload(image_angle_func_updated)
from image_angle_func_updated import *

In [3]:
# ==========================================
# Load completely new / unseen test dataset
# (No overlap with training data)
# ==========================================
new_data = pd.read_excel(r"test_data.xlsx")

# ==========================================
# Run MoveNet pose extraction on new images
# ==========================================
output_df_nd = run_movenet_multithread(
    new_data,
    url_col="image_url",   # Image URL column
    max_workers=7,         # Parallel workers
    timeout=15             # Timeout per image (seconds)
)

# Save raw pose outputs for audit/debugging
save_outputs(output_df_nd, "output_dfnd")

# ==========================================
# Generate CLIP embeddings for new images
# ==========================================
clip_embeddings_nd = run_clip_on_df(
    new_data,
    url_col="image_url",
    max_workers=7          # Keep low for torch stability
)

# ==========================================
# Apply previously trained PCA (NO refit)
# ==========================================
from sklearn.decomposition import PCA

pca_vars = 32
pca = joblib.load("pca_clip.pkl")     # Load PCA fitted on training data

# Transform new embeddings into PCA space
clip_pca = pca.transform(clip_embeddings_nd)

# ==========================================
# Build PCA feature dataframe for new data
# ==========================================
clip_pca_df = pd.DataFrame(
    clip_pca,
    columns=[f"clip_pca_{i}" for i in range(pca_vars)]
)

# Attach metadata
clip_pca_df["url"] = new_data["image_url"].values
clip_pca_df["true_angle"] = new_data["true_angle"]

# Save PCA-transformed features
save_outputs(clip_pca_df, "clip_pca_dfnd")

# ==========================================
# Merge pose features with CLIP PCA features
# (Final feature set for inference)
# ==========================================
final_df = output_df_nd.merge(
    clip_pca_df,
    on="url",
    how="left"
)

Processing: 100%|██████████████████████████████████████████████████████████████████████| 14/14 [00:06<00:00,  2.22it/s]


Saved files:
 → output_dfnd_18012026093737.csv
 → output_dfnd_18012026093737.parquet


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:07<00:00,  1.86it/s]


Saved files:
 → clip_pca_dfnd_18012026093746.csv
 → clip_pca_dfnd_18012026093746.parquet
Saved files:
 → final_df_18012026093746.csv
 → final_df_18012026093746.parquet


In [5]:
# ==========================================
# Build final model-ready feature dataframe
# ==========================================
feature_df = build_final_feature_df(
    final_df     # Combined pose + CLIP feature dataset
)

Saved files:
 → feature_df_18012026093751.csv
 → feature_df_18012026093751.parquet


In [7]:
# ==========================================
# Attach labels / metadata back to features
# ==========================================
train_test_data = feature_df.merge(
    final_df,       # Source containing labels & additional metadata
    left_on="url",  # Join key from feature dataframe
    right_on="url", # Join key from final dataframe
    how="left"      # Preserve all feature rows
)

Saved files:
 → train_test_data_18012026093753.csv
 → train_test_data_18012026093753.parquet


In [9]:
# ==========================================
# Filter rows with valid target labels
# ==========================================
model_df = (
    train_test_data[
        train_test_data["true_angle"].notna()
    ]
    .copy()
)

# Inspect dataset shape after filtering
model_df.shape

# Check class distribution of target variable
model_df["true_angle"].value_counts()

Saved files:
 → model_df_18012026093757.csv
 → model_df_18012026093757.parquet


In [11]:
# ==========================================
# Define non-feature (metadata / label) columns
# ==========================================
NON_FEATURE_COLS = ["url", "true_angle"]

# ==========================================
# Split features and target label
# ==========================================
X = model_df.drop(columns=NON_FEATURE_COLS)  # Model input features
y = model_df["true_angle"]                   # Target variable

# Sanity check on dimensions
print(X.shape, y.shape)

(14, 187) (14,)


In [13]:
import joblib

# ==========================================
# Load trained LightGBM pose classification model
# ==========================================
model = joblib.load("lgb_pose_model.pkl")

In [15]:
# ==========================================
# Run inference on completely new / unseen images
# (No train-test split, no data leakage)
# ==========================================
y_pred = model.predict(X)

# ==========================================
# Get prediction probabilities for confidence analysis
# ==========================================
y_prob = model.predict_proba(X)

# ==========================================
# Evaluate performance on unseen dataset
# (This reflects real-world generalization)
# ==========================================
print("Accuracy:", accuracy_score(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y, y_pred))

# ==========================================
# Build result dataframe for review & audit
# ==========================================
final_df = pd.DataFrame({
    "url": model_df["url"],          # Image identifier
    "actual": model_df["true_angle"],# Ground truth label
    "pred": y_pred                   # Model prediction
})

# ==========================================
# Persist datasets for offline analysis
# ==========================================
model_df.to_excel("model_data_df.xlsx", index=False)
final_df.to_excel("final_predictions.xlsx", index=False)

Accuracy: 0.0

Classification Report:
               precision    recall  f1-score   support

    B-tilted       0.00      0.00      0.00       0.0
        Back       0.00      0.00      0.00       0.0
       Front       0.00      0.00      0.00       0.0
     UNKNOWN       0.00      0.00      0.00      14.0

    accuracy                           0.00      14.0
   macro avg       0.00      0.00      0.00      14.0
weighted avg       0.00      0.00      0.00      14.0


Confusion Matrix:
 [[ 0  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]
 [ 2 10  2  0]]
